<a href="https://colab.research.google.com/github/NeeharikaG18/Project-and-Lab-/blob/main/Copy_of_AIML_III_Module_01_Lab_02_Machine_Learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
import numpy as np
from sklearn import datasets
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 33.925097276264594 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1. Effect on Validation Accuracy:

    Generally, increasing the percentage of the validation set means fewer samples are available for training. This can lead to:
        Decreased Training Accuracy: With fewer samples for training, the model might not fit the training data as well, potentially reducing training accuracy.
        Increased Validation Accuracy Variability: A larger validation set could lead to more variability in validation accuracy because it has fewer samples to generalize from, potentially making the validation accuracy less stable.
      If we decrease the percentage of the validation set (e.g., from 25% to 20% or less):

    Effect on Validation Accuracy:
        Decreasing the percentage of the validation set means more samples are available for training. This can lead to:
            Increased Training Accuracy: With more samples for training, the model might fit the training data better, potentially increasing training accuracy.
            Decreased Variability in Validation Accuracy: A smaller validation set might lead to less variability in validation accuracy because it has more samples to generalize from, potentially making the validation accuracy more stable.

    Nearest Neighbor Classifier:

    Validation accuracy tends to decrease as the percentage of the validation set increases. This is because with more data allocated to validation, there are fewer samples for training, which can lead to overfitting on the training set and reduced generalization.
    Validation accuracy is higher initially and decreases more steeply with larger validation sets.

    Random Classifier:

    Validation accuracy remains relatively stable across different percentages of the validation set. This is expected because the random classifier assigns labels randomly, and the accuracy is not heavily influenced by the size of the validation set.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Functions for nearest neighbor and random classifier
def NN1(traindata, trainlabel, query):
    diff = traindata - query
    sq = diff * diff
    dist = sq.sum(1)
    label = trainlabel[np.argmin(dist)]
    return label

def NN(traindata, trainlabel, testdata):
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

def RandomClassifier(traindata, trainlabel, testdata):
    classes = np.unique(trainlabel)
    rints = np.random.randint(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

def Accuracy(gtlabel, predlabel):
    assert len(gtlabel) == len(predlabel), "Lengths of ground-truth and predicted labels should be the same"
    correct = np.sum(gtlabel == predlabel)
    return correct / len(gtlabel)

# Function to split data
def split(data, label, percent):
    rnd = np.random.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

# Load the dataset and split into training and testing sets
from sklearn.datasets import fetch_california_housing

dataset = fetch_california_housing()
data = dataset.data
target = dataset.target.astype(int)

# Split into training and testing (80% training, 20% testing)
testdata, testlabel, alltraindata, alltrainlabel = split(data, target, 20 / 100)

# Experiment with different validation set percentages
validation_percentages = np.arange(10, 91, 5) / 100  # from 10% to 90% in steps of 5%
nn_validation_accuracies = []
random_validation_accuracies = []

for val_percentage in validation_percentages:
    # Split training data into training and validation
    traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, val_percentage)

    # Calculate validation accuracy for nearest neighbor
    valpred_nn = NN(traindata, trainlabel, valdata)
    valAccuracy_nn = Accuracy(vallabel, valpred_nn)
    nn_validation_accuracies.append(valAccuracy_nn * 100)

    # Calculate validation accuracy for random classifier
    valpred_random = RandomClassifier(traindata, trainlabel, valdata)
    valAccuracy_random = Accuracy(vallabel, valpred_random)
    random_validation_accuracies.append(valAccuracy_random * 100)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(validation_percentages * 100, nn_validation_accuracies, marker='o', linestyle='-', color='b', label='Nearest Neighbor')
plt.plot(validation_percentages * 100, random_validation_accuracies, marker='o', linestyle='-', color='r', label='Random Classifier')
plt.title('Validation Accuracy vs. Validation Set Percentage')
plt.xlabel('Percentage of Validation Set')
plt.ylabel('Validation Accuracy (%)')
plt.xticks(np.arange(10, 91, 10))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


2.
Training Set Size Impact:

    More Training Data:
         With a larger training set, the model has more data to learn from, potentially leading to better generalization and higher accuracy on both the training and validation sets.
    Less Training Data:

        Limited training data can lead to overfitting, where the model memorizes the training examples rather than learning to generalize.
 Validation Set Size Impact:

    Larger Validation Set:
        A larger validation set provides a more reliable estimate of how well the model will generalize to unseen data. It reduces variability in validation accuracy estimates and helps in detecting overfitting.   
    Smaller Validation Set:

     A smaller validation set may not accurately represent the variability in real-world data. It can lead to higher variability in validation accuracy estimates and less confidence in predicting test set performance.    

  Generally, validation accuracy tends to increase with a larger training set. This is because more training data helps in learning better models that generalize well to the validation set.
  Nearest neighbor tends to show higher validation accuracy initially but may flatten out or decrease slightly with very large training sets, indicating potential overfitting.
  Random classifier validation accuracy remains relatively stable across different training set sizes.   
  Nearest neighbor tends to show higher test accuracy compared to the random classifier, reflecting its ability to learn and generalize patterns better.
  The size of the training and validation sets directly impacts our ability to predict accuracy on the test set:
   A larger training set generally leads to better model performance and higher validation accuracy, which translates into higher test accuracy.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Functions for nearest neighbor and random classifier
def NN1(traindata, trainlabel, query):
    diff = traindata - query
    sq = diff * diff
    dist = sq.sum(1)
    label = trainlabel[np.argmin(dist)]
    return label

def NN(traindata, trainlabel, testdata):
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

def RandomClassifier(traindata, trainlabel, testdata):
    classes = np.unique(trainlabel)
    rints = np.random.randint(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

def Accuracy(gtlabel, predlabel):
    assert len(gtlabel) == len(predlabel), "Lengths of ground-truth and predicted labels should be the same"
    correct = np.sum(gtlabel == predlabel)
    return correct / len(gtlabel)

# Function to split data
def split(data, label, percent):
    rnd = np.random.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

# Load the dataset and split into training and testing sets
from sklearn.datasets import fetch_california_housing

dataset = fetch_california_housing()
data = dataset.data
target = dataset.target.astype(int)

# Experiment with different percentages of the training set (from 10% to 90%)
train_percentages = np.arange(10, 91, 5) / 100  # from 10% to 90% in steps of 5%
nn_validation_accuracies = []
random_validation_accuracies = []
test_accuracy_nn = []
test_accuracy_random = []

for train_percent in train_percentages:
    # Split into training and testing (fixed 20% test set)
    testdata, testlabel, alltraindata, alltrainlabel = split(data, target, 20 / 100)

    # Split training data further into training and validation
    traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, train_percent)

    # Calculate validation accuracy for nearest neighbor
    valpred_nn = NN(traindata, trainlabel, valdata)
    valAccuracy_nn = Accuracy(vallabel, valpred_nn)
    nn_validation_accuracies.append(valAccuracy_nn * 100)

    # Calculate validation accuracy for random classifier
    valpred_random = RandomClassifier(traindata, trainlabel, valdata)
    valAccuracy_random = Accuracy(vallabel, valpred_random)
    random_validation_accuracies.append(valAccuracy_random * 100)

    # Train on the full training set and evaluate on test set
    traindata_full = np.concatenate((traindata, valdata), axis=0)
    trainlabel_full = np.concatenate((trainlabel, vallabel), axis=0)

    # Nearest neighbor classifier test accuracy
    testpred_nn = NN(traindata_full, trainlabel_full, testdata)
    testAccuracy_nn = Accuracy(testlabel, testpred_nn)
    test_accuracy_nn.append(testAccuracy_nn * 100)

    # Random classifier test accuracy
    testpred_random = RandomClassifier(traindata_full, trainlabel_full, testdata)
    testAccuracy_random = Accuracy(testlabel, testpred_random)
    test_accuracy_random.append(testAccuracy_random * 100)

# Plotting the results
plt.figure(figsize=(12, 6))

# Plot Validation Accuracy vs. Training Set Percentage
plt.subplot(1, 2, 1)
plt.plot(train_percentages * 100, nn_validation_accuracies, marker='o', linestyle='-', color='b', label='Nearest Neighbor')
plt.plot(train_percentages * 100, random_validation_accuracies, marker='o', linestyle='-', color='r', label='Random Classifier')
plt.title('Validation Accuracy vs. Training Set Percentage')
plt.xlabel('Percentage of Training Set')
plt.ylabel('Validation Accuracy (%)')
plt.xticks(np.arange(10, 91, 10))
plt.grid(True)
plt.legend()

# Plot Test Accuracy vs. Training Set Percentage
plt.subplot(1, 2, 2)
plt.plot(train_percentages * 100, test_accuracy_nn, marker='o', linestyle='-', color='b', label='Nearest Neighbor')
plt.plot(train_percentages * 100, test_accuracy_random, marker='o', linestyle='-', color='r', label='Random Classifier')
plt.title('Test Accuracy vs. Training Set Percentage')
plt.xlabel('Percentage of Training Set')
plt.ylabel('Test Accuracy (%)')
plt.xticks(np.arange(10, 91, 10))
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


3.Choosing a good percentage for the validation set is crucial to strike a balance between effectively training the model and reliably estimating its performance on unseen data.
Desired Accuracy Estimation:

  A larger validation set provides a more reliable estimate of the model's performance on unseen data. If precise estimation of test accuracy is critical, a larger validation set percentage (e.g., 20-30%) is advisable.

  Larger datasets can afford larger validation sets without sacrificing too much training data. For smaller datasets, a larger validation set might not leave enough data for training, leading to poorer model performance.


25-30%: These percentages can be chosen if more confidence in the validation accuracy estimate is desired. They provide more data for validation, which can help reduce variability in performance estimates and better capture the model's generalization ability.
Based on the experiment results, a validation set percentage around 20-30% appears reasonable for balancing effective model training with reliable estimation of model performance.
Smaller datasets or more complex models might benefit from slightly larger validation sets, while larger datasets with simpler models might suffice with smaller validation sets.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Functions for nearest neighbor and random classifier
def NN1(traindata, trainlabel, query):
    diff = traindata - query
    sq = diff * diff
    dist = sq.sum(1)
    label = trainlabel[np.argmin(dist)]
    return label

def NN(traindata, trainlabel, testdata):
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

def RandomClassifier(traindata, trainlabel, testdata):
    classes = np.unique(trainlabel)
    rints = np.random.randint(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

def Accuracy(gtlabel, predlabel):
    assert len(gtlabel) == len(predlabel), "Lengths of ground-truth and predicted labels should be the same"
    correct = np.sum(gtlabel == predlabel)
    return correct / len(gtlabel)

# Function to split data
def split(data, label, percent):
    rnd = np.random.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

# Load the dataset and split into training and testing sets
from sklearn.datasets import fetch_california_housing

dataset = fetch_california_housing()
data = dataset.data
target = dataset.target.astype(int)

# Fixed 20% test set
testdata, testlabel, alltraindata, alltrainlabel = split(data, target, 20 / 100)

# Experiment with different percentages of the validation set
val_percentages = np.concatenate(([0.1, 1, 5, 10], np.arange(20, 101, 10))) / 100  # from 0.1% to 100% in steps of 10%
nn_validation_accuracies = []
random_validation_accuracies = []

for val_percent in val_percentages:
    # Split the training set into training and validation sets
    traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, (100 - val_percent) / 100)

    # Calculate validation accuracy for nearest neighbor
    valpred_nn = NN(traindata, trainlabel, valdata)
    valAccuracy_nn = Accuracy(vallabel, valpred_nn)
    nn_validation_accuracies.append(valAccuracy_nn * 100)

    # Calculate validation accuracy for random classifier
    valpred_random = RandomClassifier(traindata, trainlabel, valdata)
    valAccuracy_random = Accuracy(vallabel, valpred_random)
    random_validation_accuracies.append(valAccuracy_random * 100)

# Plotting the results
plt.figure(figsize=(10, 6))

# Plot Validation Accuracy vs. Validation Set Percentage
plt.plot(val_percentages * 100, nn_validation_accuracies, marker='o', linestyle='-', color='b', label='Nearest Neighbor')
plt.plot(val_percentages * 100, random_validation_accuracies, marker='o', linestyle='-', color='r', label='Random Classifier')
plt.title('Validation Accuracy vs. Validation Set Percentage')
plt.xlabel('Percentage of Validation Set')
plt.ylabel('Validation Accuracy (%)')
plt.grid(True)
plt.legend()
plt.xticks(np.arange(0, 101, 10))
plt.tight_layout()
plt.show()


> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
dataset = fetch_california_housing()
data = dataset.data
target = dataset.target.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Create KNN classifiers with n_neighbors = 1 and n_neighbors = 3
knn1 = KNeighborsClassifier(n_neighbors=1)
knn3 = KNeighborsClassifier(n_neighbors=3)

# Train the models
knn1.fit(X_train, y_train)
knn3.fit(X_train, y_train)

# Predict on the test set
y_pred_knn1 = knn1.predict(X_test)
y_pred_knn3 = knn3.predict(X_test)

# Calculate accuracy scores
accuracy_knn1 = accuracy_score(y_test, y_pred_knn1)
accuracy_knn3 = accuracy_score(y_test, y_pred_knn3)

print(f"Accuracy of KNN with 1 neighbor: {accuracy_knn1:.4f}")
print(f"Accuracy of KNN with 3 neighbors: {accuracy_knn3:.4f}")


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1. Averaging validation accuracy across multiple splits in cross-validation provides more stable and reliable results compared to a single train-test split. By evaluating the model on different subsets of the training data, it reduces variability and ensures a more accurate estimate of how well the model will perform on unseen data. This approach helps in detecting overfitting and provides a consistent measure of the model's generalization capability, enhancing confidence in its performance evaluation and suitability for real-world applications.


2.Averaging validation accuracy across multiple splits in cross-validation offers a more accurate estimate of test accuracy compared to a single train-test split. By evaluating the model on various subsets of the training data, cross-validation reduces bias and provides a robust measure of how well the model generalizes to unseen data. This approach enhances stability by smoothing out variability across different splits, ensuring a reliable assessment of the model's performance and supporting confident decisions in model selection and deployment.


3.Increasing the number of iterations in cross-validation generally leads to a more accurate estimate of test accuracy. More iterations provide a broader range of training-validation splits, reducing the impact of random chance in any single split. This results in a more stable and reliable assessment of how well the model will perform on unseen data, enhancing confidence in its generalization capability. However, while higher iterations improve accuracy estimation, they also increase computational demands. Therefore, the number of iterations chosen balances the need for accuracy with practical considerations of time and resources in model evaluation.


4.Increasing the number of iterations in cross-validation helps mitigate the challenges posed by very small training or validation datasets. By repeatedly splitting the data into different train-validation subsets, cross-validation reduces the impact of dataset size limitations on model evaluation. This approach enhances the reliability of performance metrics by averaging results across multiple splits, providing a more accurate estimate of how well the model will generalize to new, unseen data. However, while higher iterations improve accuracy estimation and robustness, they also increase computational demands. Therefore, the number of iterations chosen should strike a balance between achieving reliable results and practical considerations of time and resources.


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

The accuracy of the 3 nearest neighbor (3-NN) classifier generally becomes more stable and reliable as the number of splits in cross-validation increases
Effect of Number of Splits:

  3-NN Classifier: With more splits in cross-validation, the accuracy estimate for the 3-NN classifier tends to converge to a more consistent value. This is because averaging over multiple splits reduces the variability caused by different training-validation data combinations.
  1-NN Classifier: Similarly, the accuracy of the 1-NN classifier also stabilizes with more splits, providing a more reliable estimate of its performance on unseen data. However, the accuracy might fluctuate more compared to the 3-NN classifier due to its sensitivity to individual data points.
  Effect of Split Size:

  3-NN Classifier: The split size (percentage of data used for training vs. validation) affects how well the 3-NN classifier generalizes. Larger training sets can potentially improve accuracy by providing more representative training data, while smaller validation sets might lead to overfitting or underfitting issues.
 1-NN Classifier: Similarly, the split size influences the accuracy of the 1-NN classifier. It performs well with smaller training sets due to its reliance on nearest neighbors but might benefit from larger training sets to capture more diverse patterns in the data.

 the 3-NN classifier tends to have higher accuracy compared to the 1-NN classifier because it considers more neighbors for classification, which can reduce the impact of noisy data points. The accuracy comparison between the two classifiers can vary based on dataset characteristics, split sizes, and the number of cross-validation iterations.